In [1]:
import pandas as pd 
import numpy as np

In [2]:
import torch
import os
print(torch.cuda.is_available())

True


In [3]:
os.chdir('C:\\Users\\Onur Yaman\\Tez\\data')

In [4]:
train=pd.read_csv('train_all.csv').set_index('SK_ID_CURR')

In [5]:
y=train['TARGET']


In [7]:
y.sum()

np.int64(24825)

In [165]:
train=train.select_dtypes(include=['int64', 'float64'])

In [166]:
from sklearn.model_selection import train_test_split

In [167]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix


In [168]:
def model(X,y):
    import xgboost as xgb
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Convert data into DMatrix (optimized data structure for XGBoost)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    # Set up parameters for XGBoost with GPU
    params = {
        'objective': 'binary:logistic',  # For binary classification
        'tree_method': 'gpu_hist',       # Use GPU for training
        'gpu_id': 0,                     # Specify GPU ID
        'eval_metric': 'logloss',        # Evaluation metric
        'max_depth': 3,                  # Maximum depth of a tree
        'eta': 0.1,                      # Learning rate
        'subsample': 0.8,                # Subsample ratio
        'colsample_bytree': 0.8,         # Column subsampling
        'seed': 42                       # Random seed for reproducibility
    }
    
    # Train the model
    num_rounds = 100  # Number of boosting rounds
    model = xgb.train(params, dtrain, num_rounds)
    
    # Make predictions on the test set
    y_pred = model.predict(dtest)
    y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred]  # Convert probabilities to binary predictions
    
    # Evaluate accuracy
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test, y_pred_binary)
    print("Test Accuracy:", accuracy)
    
    from sklearn.metrics import roc_auc_score
    
    y_pred_proba = model.predict(dtest)
    
    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print("ROC AUC Score:", roc_auc)
        # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred_binary)
    print("Confusion Matrix:")
    print(cm)

    return(y_test,y_pred_proba)

### SKIP 

In [12]:
import chardet

# Detect the file's encoding
with open('HomeCredit_columns_description.csv', 'rb') as f:
    result = chardet.detect(f.read())

print("Detected Encoding:", result['encoding'])
df = pd.read_csv('HomeCredit_columns_description.csv', encoding=result['encoding'])

Detected Encoding: Windows-1252


In [14]:
from sklearn.metrics import roc_auc_score

In [15]:
X_numeric=train.select_dtypes(include=['int64', 'float64'])

In [19]:
X_numeric.head(1)

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,175783.73,175783.73,219625.7,219625.7,219625.69,219625.69,1008781.2,1008781.2,4172888.2,4172888.2


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

cols_and_gini_scores=[]
for col in train.iloc[:,:-1].columns:
    print(col)

    X=train[col]
    y=y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Convert data to DMatrix (XGBoost's internal data structure)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    params = {
    'objective': 'binary:logistic',  # Binary classification
    'eval_metric': 'logloss',        # Evaluation metric
    'max_depth': 6,                  # Maximum depth of a tree
    'eta': 0.1,                      # Learning rate
    'subsample': 0.8,                # Subsample ratio
    'colsample_bytree': 0.8,         # Feature subsampling
    'seed': 42 ,
     'device':'cuda'# Random seed
}

    model = xgb.train(params=params,dtrain=dtrain)

    # Predict and evaluate
    y_pred = model.predict(dtest)

    score=(2*roc_auc_score(y_test,y_pred)-1)

    cols_and_gini_scores.append([col,score])



CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
FLAG_WORK_PHONE
FLAG_EMAIL
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
HOUR_APPR_PROCESS_START
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE
NONLIVINGAPARTMENTS_MODE
NONLIVINGAREA_MODE
APARTMENTS_MEDI
BASEMENTAREA_MEDI
YE

In [16]:
cols_and_gini_scores=pd.DataFrame(cols_and_gini_scores)

In [17]:
cols_and_gini_scores.columns=['Column','Gini']

In [18]:
cols_and_gini_scores

,Column,Gini
0,CNT_CHILDREN,0.030756
1,AMT_INCOME_TOTAL,0.046413
2,AMT_CREDIT,0.151678
3,AMT_ANNUITY,0.122968
4,AMT_GOODS_PRICE,0.165751
...,...,...
1244,client_installments_AMT_PAYMENT_mean_sum,0.083908
1245,client_installments_AMT_INSTALMENT_mean_sum,0.078004
1246,client_installments_AMT_INSTALMENT_max_sum,0.073652
1247,client_installments_AMT_PAYMENT_max_sum,0.077489


In [19]:
cols_to_drop_gini=cols_and_gini_scores[cols_and_gini_scores.Gini<0.05].Column.tolist()

In [20]:
X_gini_dropped=train.drop(cols_to_drop_gini,axis=1)

In [26]:
model(train,y)

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:27:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:27:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.9198087898151309
ROC AUC Score: 0.7666701594014009


(SK_ID_CURR
 384575    False
 214010    False
 142232    False
 389171    False
 283617    False
           ...  
 396891    False
 423085    False
 356971    False
 320581    False
 392758    False
 Name: TARGET, Length: 61503, dtype: bool,
 array([0.19513062, 0.04667897, 0.10057887, ..., 0.02131574, 0.04679579,
        0.05028156], dtype=float32))

In [29]:
model(X_gini_dropped,y)

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:27:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:27:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.9198575679235159
ROC AUC Score: 0.7655181107365145


(SK_ID_CURR
 384575    False
 214010    False
 142232    False
 389171    False
 283617    False
           ...  
 396891    False
 423085    False
 356971    False
 320581    False
 392758    False
 Name: TARGET, Length: 61503, dtype: bool,
 array([0.1906592 , 0.05047597, 0.1071021 , ..., 0.02498648, 0.05800327,
        0.05843214], dtype=float32))

In [30]:
cols_gini_greater5=cols_and_gini_scores[cols_and_gini_scores.Gini>=0.05]

In [31]:
from tqdm import tqdm

In [32]:
def correlation_elimination(df, gini_df,threshold=0.7):
    """
    Remove highly correlated columns, retaining those with higher Gini scores.

    Parameters:
        df (pd.DataFrame): Input dataset (features only).
        gini_df (pd.DataFrame): DataFrame with columns ['Column', 'Gini'].
        threshold (float): Correlation cutoff (0.8 by default).

    Returns:
        pd.DataFrame: Reduced dataset with low correlation.
        list: Names of dropped columns.
    """
    # Sort features by Gini score (descending)
    sorted_gini = gini_df.sort_values(by='Gini', ascending=False)
    
    # Get ordered column names (highest Gini first)
    sorted_columns = sorted_gini['Column'].tolist()
    
    # Filter and align columns with the dataset
    valid_columns = [col for col in sorted_columns if col in df.columns]
    df_sorted = df[valid_columns]
    
    # Calculate correlation matrix (absolute values)
    corr_matrix = df_sorted.corr().abs()
    
    # Track retained/dropped columns
    retained = []
    dropped = []
    
    # Iterate through sorted columns
    for idx, col in tqdm(enumerate(valid_columns), total=len(valid_columns), desc="Processing columns"):
        if col in dropped:
            continue
        
        # Vectorized correlation check with retained columns
        if retained:
            max_corr = corr_matrix.loc[col, retained].max()
            if max_corr > threshold:
                # Find correlated partner(s)
                correlated_with = corr_matrix.loc[col, retained][corr_matrix.loc[col, retained] > threshold].index.tolist()
                
                # Compare Gini scores (current col vs. retained partners)
                current_gini = sorted_gini.loc[sorted_gini['Column'] == col, 'Gini'].values[0]
                partner_gini = sorted_gini.loc[sorted_gini['Column'].isin(correlated_with), 'Gini'].max()
                
                if current_gini <= partner_gini:
                    dropped.append(col)
                    continue
        
        retained.append(col)
    
    # Return results
    reduced_df = df[retained]
    return reduced_df, dropped

# --------------------------------------------------
# Example Usage
# --------------------------------------------------

# Run correlation elimination
reduced_df, dropped_cols = correlation_elimination(X_gini_dropped, cols_gini_greater5, threshold=0.7)


Processing columns: 100%|███████████████████████████████████████████████████████████| 609/609 [00:00<00:00, 628.54it/s]


NameError: name 'df' is not defined

In [186]:
pwd

'C:\\Users\\Onur Yaman\\Tez\\Data'

In [53]:
X_gini_dropped.to_pickle('reduced_df_after_gini.pkl')

In [35]:
model(reduced_df,y)

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:17:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.9200039022486708
ROC AUC Score: 0.7640700930871984


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:17:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


(SK_ID_CURR
 384575    False
 214010    False
 142232    False
 389171    False
 283617    False
           ...  
 396891    False
 423085    False
 356971    False
 320581    False
 392758    False
 Name: TARGET, Length: 61503, dtype: bool,
 array([0.14261556, 0.04894163, 0.10498478, ..., 0.01865176, 0.06322792,
        0.0460953 ], dtype=float32))

In [38]:
from boruta import BorutaPy
from xgboost import XGBClassifier

In [42]:
X_boruta=reduced_df.fillna(reduced_df.median())

In [43]:
# Initialize an XGBoost classifier with GPU support
xgb = XGBClassifier(
    tree_method='gpu_hist',  # Use GPU for histogram-based tree method
    predictor='gpu_predictor',  # Use GPU for prediction
    n_estimators=100,  # Number of trees
    max_depth=5,       # Maximum depth of the tree
    random_state=42,
    n_jobs=-1          # Use all available CPU cores
)

# Initialize Boruta
boruta_selector = BorutaPy(
    estimator=xgb, 
    n_estimators='auto',  # Number of trees in XGBoost
    max_iter=50,          # Number of iterations
    random_state=42,      # Random seed
    verbose=2             # Print progress
)

# Fit Boruta on your data
boruta_selector.fit(X_boruta.values, y.values)

# Get selected features
selected_features = reduced_df.columns[boruta_selector.support_].tolist()

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	153
Rejected: 	0


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	153
Rejected: 	0


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:21:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

...

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:25:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:25:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:25:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

Iteration: 	48 / 50
Confirmed: 	39
Tentative: 	20
Rejected: 	94


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:25:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:25:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	49 / 50
Confirmed: 	41
Tentative: 	18
Rejected: 	94


BorutaPy finished running.

Iteration: 	50 / 50
Confirmed: 	41
Tentative: 	16
Rejected: 	96


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:25:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [47]:
reduced_df_boruta=reduced_df[selected_features]

In [52]:
reduced_df.to_pickle('reduced_df_after_corr_and_gini.pkl')

In [51]:
reduced_df_boruta.to_pickle('reduced_df_boruta.pkl')

In [48]:
model(reduced_df_boruta,y)

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:34:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.9198413085540543
ROC AUC Score: 0.7629078759873681


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:34:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


(SK_ID_CURR
 384575    False
 214010    False
 142232    False
 389171    False
 283617    False
           ...  
 396891    False
 423085    False
 356971    False
 320581    False
 392758    False
 Name: TARGET, Length: 61503, dtype: bool,
 array([0.18340929, 0.05035481, 0.09731816, ..., 0.02034672, 0.04947343,
        0.05002961], dtype=float32))

In [70]:
from mlxtend.feature_selection import SequentialFeatureSelector


In [73]:
def sfs_elimination(X,y):
    # Split data into training and testing sets (optional)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize an XGBoost classifier with GPU support
    xgb = XGBClassifier(
        tree_method='gpu_hist',  # Use GPU for histogram-based tree method
        predictor='gpu_predictor',  # Use GPU for prediction
        n_estimators=100,  # Number of trees
        max_depth=5,       # Maximum depth of the tree
        random_state=42,
        n_jobs=-1          # Use all available CPU cores
    )
    
    # Initialize Sequential Feature Selector
    sfs = SequentialFeatureSelector(
        estimator=xgb,
        k_features=25,  # Number of features to select
        forward=True,   # Forward selection (set to False for backward selection)
        scoring='roc_auc',  # Metric to evaluate features
        cv=3,           # Cross-validation folds
        n_jobs=-1,
        verbose=2
    )
    
    # Fit SFS on your training data
    sfs.fit(X_train.values, y_train.values)  # Use .values to pass NumPy arrays
    
    # Get the selected feature indices
    selected_feature_indices = list(sfs.k_feature_idx_)
    
    # Get the selected feature names
    selected_features = X_train.columns[selected_feature_indices]

    print("Selected Features:", selected_features.tolist())

    return(selected_features)

In [74]:
sfs_features=sfs_elimination(reduced_df_boruta,y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done  31 out of  41 | elapsed:   41.7s remaining:   13.4s
[Parallel(n_jobs=-1)]: Done  41 out of  41 | elapsed:   48.8s finished

[2025-03-01 01:16:30] Features: 1/25 -- score: 0.6567775332689049[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   39.9s remaining:   13.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   49.0s finished

[2025-03-01 01:17:19] Features: 2/25 -- score: 0.7042055015546742[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  39 | elapsed:   40.1s remaining:   15.7s
[Parallel(n_jobs=-1)]: Done  39 out of  39 | elapsed:   48.6s finished

[2025-03-01 01:18:08] Features: 3/25 -- score: 0.7154350

Selected Features: ['EXT_SOURCE_2', 'EXT_SOURCE_3', 'bureau_DAYS_CREDIT_mean', 'DAYS_EMPLOYED', 'EXT_SOURCE_1', 'DAYS_BIRTH', 'AMT_GOODS_PRICE', 'client_installments_AMT_PAYMENT_min_sum', 'bureau_DAYS_CREDIT_max', 'previous_DAYS_LAST_DUE_1ST_VERSION_sum', 'bureau_AMT_CREDIT_SUM_DEBT_mean', 'previous_NAME_CONTRACT_STATUS_Refused_mean', 'REGION_RATING_CLIENT', 'FLOORSMAX_MODE', 'client_credit_AMT_DRAWINGS_CURRENT_mean_max', 'bureau_AMT_CREDIT_SUM_max', 'previous_NAME_PORTFOLIO_POS_mean', 'bureau_AMT_CREDIT_MAX_OVERDUE_mean', 'client_credit_AMT_PAYMENT_TOTAL_CURRENT_max_mean', 'previous_NAME_YIELD_GROUP_high_mean', 'OWN_CAR_AGE', 'bureau_CREDIT_TYPE_Microloan_count_norm', 'client_cash_CNT_INSTALMENT_FUTURE_mean_max', 'previous_PRODUCT_COMBINATION_POS industry with interest_mean', 'previous_NAME_YIELD_GROUP_low_action_mean']



[2025-03-01 01:32:44] Features: 25/25 -- score: 0.7599699608380361

In [75]:
pd.DataFrame(sfs_features).to_pickle("sfs_features")

In [77]:
reduced_df_boruta[sfs_features.tolist()].to_pickle('reduced_df_boruta_sfs.pkl')

In [87]:
reduced_df_sfs=reduced_df_boruta[sfs_features.tolist()]

In [94]:
model(reduced_df_sfs.DAYS_BIRTH,y)

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:40:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.9195323805342829
ROC AUC Score: 0.5857369242376494
Confusion Matrix:
[[56554     0]
 [ 4949     0]]


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:40:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


(SK_ID_CURR
 384575    False
 214010    False
 142232    False
 389171    False
 283617    False
           ...  
 396891    False
 423085    False
 356971    False
 320581    False
 392758    False
 Name: TARGET, Length: 61503, dtype: bool,
 array([0.09315819, 0.07995021, 0.07263536, ..., 0.05705516, 0.07587265,
        0.06739827], dtype=float32))

### CONT'D

In [169]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb


In [33]:
df=pd.read_pickle('reduced_df.pkl')

In [27]:
df.fillna(0,inplace=True)

In [194]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.5, random_state=42)

In [195]:
# Standardize numerical features (important for SMOTE)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [196]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to the training data
smote = SMOTE(sampling_strategy=0.5,random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [181]:
from imblearn.over_sampling import BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [182]:
from imblearn.combine import SMOTEENN
# Apply SMOTEENN on the training set
smoteenn = SMOTEENN(random_state=42)
X_train_resampled, y_train_resampled = smoteenn.fit_resample(X_train, y_train)


KeyboardInterrupt



In [28]:
def model_smote(X_train,y_train,X_test,y_test):
    import xgboost as xgb    
    # Convert data into DMatrix (optimized data structure for XGBoost)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    # Set up parameters for XGBoost with GPU
    params = {
        'objective': 'binary:logistic',  # For binary classification
        'tree_method': 'gpu_hist',       # Use GPU for training
        'gpu_id': 0,                     # Specify GPU ID
        'eval_metric': 'auc',
        'n_estimators':250,# Evaluation metric
        'max_depth': 3,                  # Maximum depth of a tree
        'eta': 0.1,                      # Learning rate
        'subsample': 0.8,                # Subsample ratio
        'colsample_bytree': 0.8,         # Column subsampling
        'seed': 42                       # Random seed for reproducibility
    }
    
    # Train the model
    num_rounds = 100  # Number of boosting rounds
    model = xgb.train(params, dtrain, num_rounds)
    
    # Make predictions on the test set
    y_pred = model.predict(dtest)
    y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred]  # Convert probabilities to binary predictions
    
    # Evaluate accuracy
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test, y_pred_binary)
    print("Test Accuracy:", accuracy)
    
    from sklearn.metrics import roc_auc_score
    
    y_pred_proba = model.predict(dtest)
    
    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print("ROC AUC Score:", roc_auc)

    from sklearn.metrics import average_precision_score

    # y_true: True labels (0 or 1)
    # y_scores: Model's predicted probabilities for the positive class
    pr_auc = average_precision_score(y_test, y_pred_proba)
    
    print(f"PR-AUC Score: {pr_auc:.4f}")

        # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred_binary)
    print("Confusion Matrix:")
    print(pd.DataFrame(cm).head(5))

    return(y_test,y_pred_proba)

In [36]:
df.fillna(0,inplace=True)

In [37]:
df

,EXT_SOURCE_2,EXT_SOURCE_3,bureau_DAYS_CREDIT_mean,DAYS_EMPLOYED,EXT_SOURCE_1,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_DAYS_CREDIT_ENDDATE_sum,DAYS_BIRTH,AMT_GOODS_PRICE,client_installments_AMT_PAYMENT_min_sum,...,HOUR_APPR_PROCESS_START,previous_PRODUCT_COMBINATION_Cash X-Sell: high_sum,client_installments_AMT_PAYMENT_mean_min,client_credit_AMT_CREDIT_LIMIT_ACTUAL_min_sum,client_cash_NAME_CONTRACT_STATUS_Active_sum_min,previous_PRODUCT_COMBINATION_POS industry with interest_sum,bureau_CNT_CREDIT_PROLONG_max,previous_NAME_GOODS_CATEGORY_Consumer Electronics_mean,client_bureau_balance_STATUS_C_count_norm_max,client_credit_AMT_CREDIT_LIMIT_ACTUAL_max_sum
0,0.262949,0.139376,-874.0000,-637,0.083037,0.750000,-2094.0,-9461,351000.0,175783.73,...,10,0.0,11559.2470,0.0,19.0,0.0,0.0,0.000000,0.812500,0.0
1,0.622246,0.000000,-1400.7500,-1188,0.311267,0.750000,-2178.0,-16765,1129500.0,1154108.20,...,11,0.0,6731.1150,0.0,7.0,1.0,0.0,0.333333,0.000000,0.0
2,0.555912,0.729567,-867.0000,-225,0.000000,1.000000,-977.0,-19046,135000.0,16071.75,...,9,0.0,7096.1550,0.0,3.0,0.0,0.0,0.000000,0.000000,0.0
3,0.650442,0.000000,0.0000,-3039,0.000000,0.000000,0.0,-19005,297000.0,994476.70,...,17,1.0,5018.2646,1620000.0,2.0,1.0,0.0,0.000000,0.000000,1620000.0
4,0.322738,0.000000,-1149.0000,-3038,0.000000,1.000000,-783.0,-19932,513000.0,483756.38,...,11,0.0,1833.0391,0.0,10.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.681632,0.000000,0.0000,-236,0.145570,0.000000,0.0,-9327,225000.0,46241.37,...,15,0.0,7492.9243,0.0,7.0,0.0,0.0,0.000000,0.000000,0.0
307507,0.115992,0.000000,0.0000,365243,0.000000,0.000000,0.0,-20775,225000.0,60281.28,...,8,0.0,10069.8670,0.0,6.0,0.0,0.0,1.000000,0.000000,0.0
307508,0.535722,0.218859,-867.5000,-7921,0.744026,0.500000,1122.0,-14966,585000.0,41805.99,...,9,0.0,2772.6300,0.0,3.0,0.0,0.0,0.000000,0.612903,0.0
307509,0.514163,0.661024,-1104.0000,-4786,0.000000,1.000000,-859.0,-11961,319500.0,194556.81,...,9,0.0,2296.4400,0.0,9.0,0.0,0.0,0.000000,0.783784,0.0


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score,f1_score,recall_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from imblearn.over_sampling import BorderlineSMOTE

def model(X,y):
    # Train-test split (stratified)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Standardize numerical features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    ### Train XGBoost WITHOUT SMOTE
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "tree_method": "gpu_hist",  # Use GPU
        "predictor": "gpu_predictor",
        "learning_rate": 0.05,
        "max_depth": 6,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "random_state": 42
    }
    
    # Train XGBoost without SMOTE
    model_no_smote = xgb.train(params, dtrain, num_boost_round=200)
    
    # Predict on test set
    y_pred_proba_no_smote = model_no_smote.predict(dtest)
    y_pred_binary_no_smote = [1 if p > 0.5 else 0 for p in y_pred_proba_no_smote]  # Convert probabilities to binary predictions

    # Evaluate performance (No SMOTE)
    roc_auc_no_smote = roc_auc_score(y_test, y_pred_proba_no_smote)
    pr_auc_no_smote = average_precision_score(y_test, y_pred_proba_no_smote)
    recall_no_smote=recall_score(y_test, y_pred_binary_no_smote)
    f1_no_smote=f1_score(y_test, y_pred_binary_no_smote)
    
    
    ### Apply SMOTE
    smote = SMOTE(sampling_strategy=1, random_state=42,k_neighbors=10)  # Fully balance classes
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    # Train XGBoost WITH SMOTE
    dtrain_resampled = xgb.DMatrix(X_train_resampled, label=y_train_resampled)
    model_with_smote = xgb.train(params, dtrain_resampled, num_boost_round=200)
    
    # Predict on test set
    y_pred_proba_with_smote = model_with_smote.predict(dtest)
    y_pred_binary_with_smote = [1 if p > 0.5 else 0 for p in y_pred_proba_with_smote]  # Convert probabilities to binary predictions

    # Evaluate performance (With SMOTE)
    roc_auc_with_smote = roc_auc_score(y_test, y_pred_proba_with_smote)
    pr_auc_with_smote = average_precision_score(y_test, y_pred_proba_with_smote)
    recall_smote=recall_score(y_test, y_pred_binary_with_smote)
    f1_smote=f1_score(y_test, y_pred_binary_with_smote)    

    
    # Print comparison
    print(f"ROC-AUC (No SMOTE): {roc_auc_no_smote:.4f}")
    print(f"PR-AUC (No SMOTE): {pr_auc_no_smote:.4f}")
    print(f"Recall (No SMOTE): {recall_no_smote:.4f}")
    print(f"F1 (No SMOTE): {f1_no_smote:.4f}")
    print(f"ROC-AUC (With SMOTE): {roc_auc_with_smote:.4f}")
    print(f"PR-AUC (With SMOTE): {pr_auc_with_smote:.4f}")
    print(f"Recall (With SMOTE): {recall_smote:.4f}")
    print(f"F1 (With SMOTE): {f1_smote:.4f}")    


    return([roc_auc_no_smote,pr_auc_no_smote,roc_auc_with_smote,pr_auc_with_smote])

In [51]:
model(df,y)

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:10:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:10:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:10:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

ROC-AUC (No SMOTE): 0.7711
PR-AUC (No SMOTE): 0.2593
Recall (No SMOTE): 0.0285
F1 (No SMOTE): 0.0542
ROC-AUC (With SMOTE): 0.7452
PR-AUC (With SMOTE): 0.2162
Recall (With SMOTE): 0.0428
F1 (With SMOTE): 0.0774


[np.float64(0.7710726329215413),
 np.float64(0.25933650016793686),
 np.float64(0.745212333158898),
 np.float64(0.21623623868849398)]

In [199]:
model_smote(X_train_resampled,y_train_resampled,X_test,y_test)

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:38:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:38:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:38:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost

Test Accuracy: 0.9155284997008247
ROC AUC Score: 0.7201454475079708
PR-AUC Score: 0.1853
Confusion Matrix:
        0     1
0  140309  1015
1   11973   459


(SK_ID_CURR
 384575    False
 214010    False
 142232    False
 389171    False
 283617    False
           ...  
 334926    False
 205688    False
 257666     True
 129280    False
 114548    False
 Name: TARGET, Length: 153756, dtype: bool,
 array([0.30758387, 0.17459778, 0.24221593, ..., 0.29150057, 0.05328859,
        0.10165679], dtype=float32))

In [207]:
df_boruta_sfs

,EXT_SOURCE_2,EXT_SOURCE_3,bureau_DAYS_CREDIT_mean,DAYS_EMPLOYED,EXT_SOURCE_1,bureau_CREDIT_ACTIVE_Closed_count_norm,DAYS_BIRTH,AMT_GOODS_PRICE,client_installments_AMT_PAYMENT_min_sum,bureau_DAYS_CREDIT_max,...,previous_NAME_YIELD_GROUP_high_mean,OWN_CAR_AGE,bureau_CREDIT_TYPE_Microloan_count_norm,client_cash_CNT_INSTALMENT_FUTURE_mean_max,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_AMT_CREDIT_SUM_OVERDUE_mean,client_installments_NUM_INSTALMENT_NUMBER_max_min,previous_PRODUCT_COMBINATION_POS industry with interest_mean,previous_NAME_YIELD_GROUP_low_action_mean,previous_PRODUCT_COMBINATION_Cash X-Sell: high_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0.262949,0.139376,-874.0000,-637,0.083037,0.750000,-9461,351000.0,175783.73,-103.0,...,0.000000,NaN,0.0,15.000,0.0,0.0,19.0,0.000000,0.0,0.0
100003,0.622246,NaN,-1400.7500,-1188,0.311267,0.750000,-16765,1129500.0,1154108.20,-606.0,...,0.000000,NaN,0.0,7.875,0.0,0.0,6.0,0.333333,0.0,0.0
100004,0.555912,0.729567,-867.0000,-225,NaN,1.000000,-19046,135000.0,16071.75,-408.0,...,0.000000,26.0,0.0,2.250,0.0,0.0,3.0,0.000000,0.0,0.0
100006,0.650442,NaN,NaN,-3039,NaN,NaN,-19005,297000.0,994476.70,NaN,...,0.222222,NaN,NaN,16.000,NaN,NaN,1.0,0.111111,0.0,1.0
100007,0.322738,NaN,-1149.0000,-3038,NaN,1.000000,-19932,513000.0,483756.38,-1149.0,...,0.500000,NaN,0.0,18.500,0.0,0.0,10.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0.681632,NaN,NaN,-236,0.145570,NaN,-9327,225000.0,46241.37,NaN,...,1.000000,NaN,NaN,4.375,NaN,NaN,7.0,0.000000,0.0,0.0
456252,0.115992,NaN,NaN,365243,NaN,NaN,-20775,225000.0,60281.28,NaN,...,0.000000,NaN,NaN,3.000,NaN,NaN,6.0,0.000000,0.0,0.0
456253,0.535722,0.218859,-867.5000,-7921,0.744026,0.500000,-14966,585000.0,41805.99,-713.0,...,1.000000,NaN,0.0,3.000,0.0,0.0,4.0,0.000000,0.0,0.0


In [24]:
df_boruta_sfs['TARGET']=y

In [26]:
df_boruta_sfs['TARGET']=df_boruta_sfs['TARGET']*1

In [27]:
df_boruta_sfs

,EXT_SOURCE_2,EXT_SOURCE_3,bureau_DAYS_CREDIT_mean,DAYS_EMPLOYED,EXT_SOURCE_1,DAYS_BIRTH,AMT_GOODS_PRICE,client_installments_AMT_PAYMENT_min_sum,bureau_DAYS_CREDIT_max,previous_DAYS_LAST_DUE_1ST_VERSION_sum,...,previous_NAME_PORTFOLIO_POS_mean,bureau_AMT_CREDIT_MAX_OVERDUE_mean,client_credit_AMT_PAYMENT_TOTAL_CURRENT_max_mean,previous_NAME_YIELD_GROUP_high_mean,OWN_CAR_AGE,bureau_CREDIT_TYPE_Microloan_count_norm,client_cash_CNT_INSTALMENT_FUTURE_mean_max,previous_PRODUCT_COMBINATION_POS industry with interest_mean,previous_NAME_YIELD_GROUP_low_action_mean,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0.262949,0.139376,-874.0000,-637,0.083037,-9461,351000.0,175783.73,-103.0,125.0,...,1.000000,1681.029,NaN,0.000000,NaN,0.0,15.000,0.000000,0.0,1
100003,0.622246,NaN,-1400.7500,-1188,0.311267,-16765,1129500.0,1154108.20,-606.0,-3013.0,...,0.666667,0.000,NaN,0.000000,NaN,0.0,7.875,0.333333,0.0,0
100004,0.555912,0.729567,-867.0000,-225,NaN,-19046,135000.0,16071.75,-408.0,-694.0,...,1.000000,0.000,NaN,0.000000,26.0,0.0,2.250,0.000000,0.0,0
100006,0.650442,NaN,NaN,-3039,NaN,-19005,297000.0,994476.70,NaN,366336.0,...,0.222222,NaN,0.0,0.222222,NaN,NaN,16.000,0.111111,0.0,0
100007,0.322738,NaN,-1149.0000,-3038,NaN,-19932,513000.0,483756.38,-1149.0,-4186.0,...,0.333333,0.000,NaN,0.500000,NaN,0.0,18.500,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0.681632,NaN,NaN,-236,0.145570,-9327,225000.0,46241.37,NaN,0.0,...,1.000000,NaN,NaN,1.000000,NaN,NaN,4.375,0.000000,0.0,0
456252,0.115992,NaN,NaN,365243,NaN,-20775,225000.0,60281.28,NaN,-2316.0,...,1.000000,NaN,NaN,0.000000,NaN,NaN,3.000,0.000000,0.0,0
456253,0.535722,0.218859,-867.5000,-7921,0.744026,-14966,585000.0,41805.99,-713.0,-4438.0,...,1.000000,NaN,NaN,1.000000,NaN,0.0,3.000,0.000000,0.0,0


In [39]:
reduced_df=pd.concat([reduced_df,train_numeric['TARGET']],axis=1)

In [42]:
X=train_numeric.iloc[:,:-1]
y=train_numeric['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert data to DMatrix (XGBoost's internal data structure)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
'objective': 'binary:logistic',  # Binary classification
'eval_metric': 'logloss',        # Evaluation metric
'max_depth': 6,                  # Maximum depth of a tree
'eta': 0.1,                      # Learning rate
'subsample': 0.8,                # Subsample ratio
'colsample_bytree': 0.8,         # Feature subsampling
'seed': 42 ,
 'device':'cuda'# Random seed
}

model = xgb.train(params=params,dtrain=dtrain)

# Predict and evaluate
y_pred = model.predict(dtest)

score=(2*roc_auc_score(y_test,y_pred)-1)


In [43]:
score

np.float64(0.47423774128175866)

In [44]:
train_numeric

,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,REGION_RATING_CLIENT,...,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum,TARGET
0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,NaN,2,...,175783.73,219625.700,219625.700,219625.690,219625.690,1.008781e+06,1.008781e+06,4.172888e+06,4.172888e+06,1
1,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,NaN,1,...,1154108.20,1150977.400,1150977.400,1618864.800,1618864.800,4.394102e+06,4.394102e+06,1.134881e+07,1.134881e+07,0
2,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,26.0,2,...,16071.75,21288.465,21288.465,21288.465,21288.465,3.172190e+04,3.172190e+04,6.386539e+04,6.386539e+04,0
3,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,NaN,2,...,994476.70,691786.900,691786.900,1007153.400,1007153.400,1.057860e+06,1.057860e+06,3.719995e+06,3.719995e+06,0
4,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,NaN,2,...,825845.80,280199.700,294631.120,806127.940,835985.300,8.367034e+05,8.367034e+05,1.139621e+07,1.179154e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,-1982,NaN,1,...,46241.37,52450.470,52450.470,52450.470,52450.470,8.970507e+04,8.970507e+04,3.671533e+05,3.671533e+05,0
307507,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,-4090,NaN,2,...,60281.28,60419.203,60419.203,60419.203,60419.203,6.044679e+04,6.044679e+04,3.625152e+05,3.625152e+05,0
307508,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,-5150,NaN,3,...,61529.04,33413.758,33413.758,57622.812,61595.906,6.166175e+04,6.166175e+04,3.048920e+05,3.247575e+05,0
307509,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,-931,NaN,2,...,194556.81,171592.420,171592.420,194556.810,194556.810,1.945568e+05,1.945568e+05,1.773976e+06,1.773976e+06,1


In [19]:
import torch

In [21]:
print(torch.cuda.is_available())


False


In [24]:
!python -V

Python 3.11.7
